In [1]:
import pandas as pd
import numpy as np
import requests
import billboard
import csv
import bs4 as bs
import urllib.request
import re

In [2]:
df = pd.read_csv('/Users/joshuakowal/DSCI 303 Assignments/spotify_hits/data/tracks_w_lyrics_en.csv')
df.head()

,Unnamed: 0,track_id,track_name,track_artist,lyrics,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,language
0,1,004s3t0ONYlzxII9PLgU6z,I Feel Alive,Steady Rollin,"The trees, are singing in the wind The sky blu...",28,3z04Lb9Dsilqw68SHt6jLB,Love & Loss,2017-11-21,Hard Rock Workout,...,-4.739,1,0.0442,0.01170,0.00994,0.3470,0.404,135.225,373512,en
1,2,00chLpzhgVjxs1zKC9UScL,Poison,Bell Biv DeVoe,"NA Yeah, Spyderman and Freeze in full effect U...",0,6oZ6brjB8x3GoeSYdwJdPc,Gold,2005-01-01,"Back in the day - R&B, New Jack Swing, Swingbe...",...,-7.504,0,0.2160,0.00432,0.00723,0.4890,0.650,111.904,262467,en
2,3,00cqd6ZsSkLZqGMlQCR0Zo,Baby It's Cold Outside (feat. Christina Aguilera),CeeLo Green,I really can't stay Baby it's cold outside I'v...,41,3ssspRe42CXkhPxdc12xcp,CeeLo's Magic Moment,2012-10-29,Christmas Soul,...,-5.819,0,0.0341,0.68900,0.00000,0.0664,0.405,118.593,243067,en
3,4,00emjlCv9azBN0fzuuyLqy,Dumb Litty,KARD,Get up out of my business You don't keep me fr...,65,7h5X3xhh3peIK9Y0qI5hbK,KARD 2nd Digital Single ‘Dumb Litty’,2019-09-22,K-Party Dance Mix,...,-1.993,1,0.0409,0.03700,0.00000,0.1380,0.240,130.018,193160,en
4,5,00f9VGHfQhAHMCQ2bSjg3D,Soldier,James TW,"Hold your breath, don't look down, keep trying...",70,3GNzXsFbzdwM0WKCZtgeNP,Chapters,2019-04-26,urban contemporary,...,-6.157,1,0.0550,0.28000,0.00000,0.0975,0.305,147.764,224720,en


In [3]:
#Remove ' (feat. ...)'

for i in range(df.shape[0]):
    name = df.iloc[i,2]
    
    #Get rid of featured
    if ' (' in name:
        name = name.split(' (')[0]
        
    #Get rid of - remastered
    if ' - ' in name:
        name = name.split(' - ')[0]
        
    df.iloc[i,2] = name
    
    #Error Checking
    assert ' (feat.' not in df.iloc[i,2]
    assert '(feat.' not in df.iloc[i,2]
    assert ' - ' not in df.iloc[i,2]
    
df.head()

,Unnamed: 0,track_id,track_name,track_artist,lyrics,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,language
0,1,004s3t0ONYlzxII9PLgU6z,I Feel Alive,Steady Rollin,"The trees, are singing in the wind The sky blu...",28,3z04Lb9Dsilqw68SHt6jLB,Love & Loss,2017-11-21,Hard Rock Workout,...,-4.739,1,0.0442,0.01170,0.00994,0.3470,0.404,135.225,373512,en
1,2,00chLpzhgVjxs1zKC9UScL,Poison,Bell Biv DeVoe,"NA Yeah, Spyderman and Freeze in full effect U...",0,6oZ6brjB8x3GoeSYdwJdPc,Gold,2005-01-01,"Back in the day - R&B, New Jack Swing, Swingbe...",...,-7.504,0,0.2160,0.00432,0.00723,0.4890,0.650,111.904,262467,en
2,3,00cqd6ZsSkLZqGMlQCR0Zo,Baby It's Cold Outside,CeeLo Green,I really can't stay Baby it's cold outside I'v...,41,3ssspRe42CXkhPxdc12xcp,CeeLo's Magic Moment,2012-10-29,Christmas Soul,...,-5.819,0,0.0341,0.68900,0.00000,0.0664,0.405,118.593,243067,en
3,4,00emjlCv9azBN0fzuuyLqy,Dumb Litty,KARD,Get up out of my business You don't keep me fr...,65,7h5X3xhh3peIK9Y0qI5hbK,KARD 2nd Digital Single ‘Dumb Litty’,2019-09-22,K-Party Dance Mix,...,-1.993,1,0.0409,0.03700,0.00000,0.1380,0.240,130.018,193160,en
4,5,00f9VGHfQhAHMCQ2bSjg3D,Soldier,James TW,"Hold your breath, don't look down, keep trying...",70,3GNzXsFbzdwM0WKCZtgeNP,Chapters,2019-04-26,urban contemporary,...,-6.157,1,0.0550,0.28000,0.00000,0.0975,0.305,147.764,224720,en


# Get Billboard Top 100 for Years 1990-2019

In [4]:
all_years = pd.DataFrame() 
for year in range(1970,2019):
    if year < 2015:
        url = "http://tsort.info/music/yr" + str(year) + ".htm"
        page = requests.get(url)
        soup = bs.BeautifulSoup(page.content, 'html.parser')
        tables = soup.findChildren("table")
        song_table = tables[0]
        rows = song_table.findChildren(['th', 'tr'])
        all_rows = []
        for row in rows:
            cells = row.findChildren('td')
            for cell in cells:
                value = cell.string
                all_rows.append(value)
        relevant = all_rows[all_rows.index("1"):]
        artist = relevant[1::5]
        songName = relevant[2::5]
        years = relevant[3::5]
        years.append("nan")
        dfdict = {"artist":artist, "songName":songName, "year":years}
        print(year)
        d = pd.DataFrame(dfdict)
        all_years = all_years.append(d)
    else:
        chart = billboard.ChartData('hot-100-songs', year = year)
        lines = str(chart).split('\n')[2:]
        art = []
        sn = []
        years = []
        print(year)
        for line in lines:
            title = line.split('. ')[1].split(' by')[0]
    
            main_art = line.split(' by ')[1]
            art.append(main_art)
            sn.append(title)
            years.append(year)
        d = pd.DataFrame({"artist":art, "songName":sn, "year":years})
        all_years = all_years.append(d)

1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018


In [5]:
all_years.head()

,artist,songName,year
0,The Beatles,Let it Be,1970
1,Mungo Jerry,In the Summertime,1970
2,Shocking Blue,Venus,1970
3,Simon & Garfunkel,Bridge Over Troubled Water,1970
4,B J Thomas,Raindrops Keep Falling On My Head,1970


In [6]:
songs = list(all_years['songName'].str.lower())
df['Hit or Not'] = 0
nrows, ncols = df.shape
for i in range(nrows):
    song = df.iloc[i,2].lower()
    if song in songs:
        df.loc[i,'Hit or Not'] = 1

In [7]:
t = list(df['Hit or Not'])
count = 0
for tt in t:
    if tt == 1:
        count += 1
count

3679

In [9]:
df.to_csv('/Users/joshuakowal/DSCI 303 Assignments/spotify_hits/data/tracks_w_lyrics_w_hits_en.csv')